# Crush Rig Lab
Written by Matt MacDonald for CIGITI at the Hospital for Sick Children Toronto
***

All tools to manipulate data will be obtained from the crush_plot.py file.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib notebook
# import matplotlib as mpl
# mpl.rcParams['savefig.dpi'] = 120
# mpl.rcParams['figure.dpi'] = 120

In [ ]:
from crush_plot import *

The crush data must be collected using the crush rig and crush.py and stored in the expected folder structure at the root directory indicated by PATH.

In [ ]:
#PATH = Path('')

Load all data and modify as needed.

In [ ]:
study = study_outline(PATH)
crushes = study_data(study)
crushes = modify(crushes)

In [ ]:
study

In [ ]:
crushes

Time series plots can be made with time_plot(crushes) and stress-strain plots can be made with stress_plot(crushes).

In [ ]:
time_plot(crushes)

In [ ]:
stress_plot(crushes)

Select subsets of the crushes using the convenience functions:
- patient(*patient*)
- tissue(*tissue*)
- protocol(*protocol*)
- load(*load*)

In [ ]:
# Convenience functions
def patient(patient):
    return crushes.loc[crushes['Patient'] == patient.upper(), :]

def tissue(tissue):
    return crushes.loc[crushes['Tissue'] == tissue.upper(), :]

def protocol(protocol):
    return crushes.loc[crushes['Protocol'] == protocol.lower(), :]

def load(load):
    return crushes.loc[crushes['Load'] == load.lower(), :]

Subsets can be combined using select( ).

In [ ]:
def select(*args, **kwargs):
    args = list(args)
    for key, val in kwargs.items():
        args.append(globals()[key](val))
    if len(args) == 1:
        return args[0]
    if len(args) > 2:
        args = [args[0], select(*args[1:])]
    index = (args[0].index).intersection(args[1].index)
    return crushes.loc[index, :]

In [ ]:
protocol('hold')

In [ ]:
select(patient('pc3'), protocol('hold'), load('1200g'))

In [ ]:
x = select(tissue('sb'), load('200g'))
x

In [ ]:
stress_plot(x)

In [ ]:
time_plot(select(tissue='sb', load='800g'))

In [ ]:
x = select(protocol='stop', load='1200g')
x

In [ ]:
time_plot(x)

In [ ]:
time_plot(select(patient('pc3'), protocol='stop'))